In [3]:
from jinja2 import Template

# Step 1: 读取模板内容
with open('/home/tangziyi/RD-Agent/rdagent/components/coder/factor_coder/template_debug.jinjia2', 'r') as f:
    template_content = f.read()

# Step 2: 渲染模板
template = Template(template_content)
rendered_code = template.render(
    expression="(0.3 * $pettm) + (0.2 * $pbmrq) + (0.2 * $turn) + (0.3 * (($close - $open) > 0 ? 1 : 0))", # "DELAY($high + $low / 2, 5)",
    factor_name="FACTOR_1"
    )

# Step 3: 打印渲染后的代码
print(rendered_code)
exec(rendered_code)


import pandas as pd
import numpy as np
import os
from rdagent.components.coder.factor_coder.expr_parser import parse_expression, parse_symbol
from rdagent.components.coder.factor_coder.function_lib import (
    ABS, ADD, AND, CORR, COUNT, COVARIANCE, DECAYLINEAR, DELAY, DELTA, DIVIDE, 
    EMA, EXP, FILTER, HIGHDAY, LOG, LOWDAY, MAX, MEAN, MEDIAN, MIN, MULTIPLY, 
    OR, POW, PROD, RANK, REGBETA, REGRESI, SEQUENCE, SIGN, SMA, SQRT, STD, 
    SUBTRACT, SUM, SUMAC, SUMIF, TS_MAX, TS_MIN, TS_RANK, TS_ZSCORE, WMA, ZSCORE
)


def calculate_factor(expr: str, name: str):
    # Stock DataFrame
    # df.columns: ['$open', '$close', '$high', '$low', '$volume', '$amount', '$turn', '$pettm', '$pbmrq']
    df = pd.read_hdf('/home/tangziyi/RD-Agent/rdagent/scenarios/qlib/experiment/factor_data_template/daily_pv_all.h5', key='data')
    
    expr = parse_symbol(expr, df.columns)
    expr = parse_expression(expr)

    # replace 'var_name' by 'df['$var_name']
    for col in df.columns:
        expr = 